In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics                 import confusion_matrix
from sklearn.metrics                 import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

In [2]:
dta=pd.read_csv('/Users/nihz415/Desktop/benekiva/p2/test.csv')
dta
dta.columns

Index(['job_title', 'Persona'], dtype='object')

In [3]:
##clean
def clean_text(text):
    text=re.sub(r',',' ' ,text)
    return text
dta['tidy'] = dta['job_title'].astype('U').apply(clean_text)  
dta['tidy']=dta['tidy'].str.lower()   
dta['tidy'] = dta['tidy'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

def classify(text):
    if text=='IT':
        clf=2
    else:        
        if text=='Claims':
            clf=1
        else:
            if text=='Executive':
                clf=3
            else:
                clf=0
    return (clf)                
dta['target'] = dta['Persona'].astype('U').apply(classify)                 

dta1=dta[['tidy','target']]

In [4]:
###undersampling
pclass1 = dta1[dta1.target==1]
pclass2 = dta1[dta1.target==2]
pclass3 = dta1[dta1.target==3]
count1, count2, count3 = dta1.target.value_counts()
print(count1)
print(count2)
print(count3)


under0 = pclass1.sample(count3,replace=True)
under1 = pclass2.sample(count3,replace=True)
dta2 = pd.concat([under0,under1,pclass3],axis=0)
count1, count2, count3 = dta2.target.value_counts()
print(count1)
print(count2)
print(count3)

947
443
401
401
401
401


# With Monte-Carlo, nmc=100 for average

In [8]:
####turn into corpus， set X and y
corpus= dta2['tidy'].to_list()   
vectorizer_count= CountVectorizer(lowercase= True,ngram_range = (1,1),max_df= 0.85,min_df= 0.005);
X=vectorizer_count.fit_transform(dta2['tidy'].values.astype('U'))
features_frequency= pd.DataFrame({'feature':vectorizer_count.get_feature_names(),'feature_frequency' : X.toarray().sum(axis=0)})
X.shape

y = dta2.target

In [9]:
#########logistic
lc = LogisticRegression(max_iter=1000,penalty="none")
# Set up randomized cross validation using standard Scikit-learn tools
shuffle = ShuffleSplit(n_splits=100, test_size=.25)
CVInfo = cross_validate(lc, X, y, cv=shuffle,return_train_score=True,n_jobs=-1)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.9734700665188473
0.933654485049834


In [10]:
#########knn
training_accuracy = []
test_accuracy = []
# try n_neighbors from 1 to 25
neighbors_settings = range(10, 110,10)
shuffle = ShuffleSplit(n_splits=100, test_size=.25)

for n_neighbors in neighbors_settings:
    # build the model
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    CVInfo = cross_validate(knn, X, y, cv=shuffle,return_train_score=True,n_jobs=-1)
    # record train accuracy
    training_accuracy.append(np.mean(CVInfo['train_score']))
    # record test accuracy
    test_accuracy.append(np.mean(CVInfo['test_score']))

nbest = np.argmax(test_accuracy)
best_neighbors = neighbors_settings[nbest]
print("best_neighbors=",best_neighbors)
print(test_accuracy)

best_neighbors= 50
[0.9575747508305648, 0.9586378737541529, 0.9578405315614619, 0.9599999999999997, 0.9608970099667773, 0.9607641196013287, 0.9598671096345512, 0.9588039867109635, 0.9591029900332227, 0.9603322259136212]


In [11]:
knn = KNeighborsClassifier(n_neighbors=best_neighbors)
X_train, X_test, y_train, y_test = train_test_split( X, y )
knn.fit(X_train,y_train)
print(knn.score(X_train,y_train))
print(knn.score(X_test,y_test))

0.9645232815964523
0.9568106312292359


# Without Montecarlo

In [12]:
###################
dta3=dta2[['tidy','target']]
dta3['ML_group']  = np.random.randint(10,size=dta3.shape[0])
dta3['ML_group']  = (dta3['ML_group']<=5)*0 + (dta3['ML_group']==6)*1 + (dta3['ML_group']==7)*1+(dta3['ML_group']==9)*2+(dta3['ML_group']==8)*2
X_train = X[np.where(dta3['ML_group']==0)[0],:]
X_valid = X[np.where(dta3['ML_group']==1)[0],:]
X_test  = X[np.where(dta3['ML_group']==2)[0],:]
y_train = dta3.loc[dta3['ML_group']==0,['target']]['target'].to_numpy()
y_valid = dta3.loc[dta3['ML_group']==1,['target']]['target'].to_numpy()
y_test  = dta3.loc[dta3['ML_group']==2,['target']]['target'].to_numpy()

In [14]:
from sklearn import linear_model
def logistic_reg_classifier_mult_labels(X_train,y_train,X_valid,y_valid,X_test,y_test):
    
    ' . '
    categories         = pd.DataFrame(np.sort(np.unique(y_train))).reset_index()
    categories.columns = ['index','label']
    

    ' . '    
    ccp_train_list = []
    ccp_valid_list = []
    ccp_test_list  = []
    for cat in categories['label'].to_list():
        y_train_c = 1*(y_train==cat)
        clf       = linear_model.LogisticRegression(tol          = 0.0001,
                                                    max_iter     = 10000,
                                                    random_state = None).fit(X_train, y_train_c)
        ccp_train_list.append(  clf.predict_proba(X_train)[:,1])
        ccp_valid_list.append(  clf.predict_proba(X_valid)[:,1])
        ccp_test_list.append(   clf.predict_proba(X_test)[:,1])
    
    ' . Topic probability matrix'
    ccp_train = pd.DataFrame(ccp_train_list).transpose()
    ccp_valid = pd.DataFrame(ccp_valid_list).transpose()
    ccp_test  = pd.DataFrame(ccp_test_list).transpose()
    
    'reset column index'
    ccp_train.columns = categories['label'].to_list()
    ccp_valid.columns = categories['label'].to_list()
    ccp_test.columns = categories['label'].to_list()
    
    'Choosing your predictive category for the y '
    ccp_train['label_hat'] =  ccp_train.idxmax(axis=1)
    ccp_valid['label_hat'] =  ccp_valid.idxmax(axis=1)
    ccp_test['label_hat']  =  ccp_test.idxmax(axis=1)    
    
    'caculate'
    confusionmatrix=confusion_matrix(y_test,ccp_test['label_hat'] )
    correct =  np.trace(confusionmatrix)
    total = confusionmatrix.sum()
    percent_accuracy = correct/total
        
    return(confusionmatrix,percent_accuracy)

logistic_reg_classifier_mult_labels(X_train,y_train,X_valid,y_valid,X_test,y_test)

(array([[69,  0,  0],
        [ 0, 73,  6],
        [ 1,  5, 75]]),
 0.9475982532751092)

In [15]:
#find best k
conf_matrices = []
def find_best_k(X_train,X_valid,y_train,y_valid):
    
    k            = 1;
    results_list = [];
    max_k_nn     = 100
    
    for k in range(1,max_k_nn+1):
       clf         = KNeighborsClassifier(n_neighbors=k).fit(X_train , y_train )
       y_hat_valid = clf.predict(X_valid)
       conf_matrices.append(confusion_matrix(y_valid, y_hat_valid ))
   
    for i in range(0,max_k_nn):
       confusionmatrix=conf_matrices[i]
       correct =  np.trace(confusionmatrix)
       total = confusionmatrix.sum()
       percent_accuracy = correct/total
       result_for_i=[i+1,percent_accuracy]
       results_list.append(result_for_i)
            
    for u in range(1,max_k_nn):
        if results_list[u][1]>results_list[u-1][1]:
            best=results_list[u]
            
    return best

#find_best_k(X_train,X_valid,y_train,y_valid)
best=find_best_k(X_train,X_valid,y_train,y_valid)[0]

In [16]:
def K_NN_clf_result(best,X_train,y_train,X_valid,y_valid,X_test,y_test):
    
    best=find_best_k(X_train,X_valid,y_train,y_valid)[0]
    
    Y_hat_train = []
    Y_hat_valid = []
    Y_hat_test  = []
    
    clf = KNeighborsClassifier(n_neighbors=best).fit(X_train , y_train )
    Y_hat_train.append(clf.predict(X_train))
    Y_hat_valid.append(clf.predict(X_valid))
    Y_hat_test.append(clf.predict(X_test))
    
    Y_hat_train = pd.DataFrame(Y_hat_train).transpose()
    Y_hat_valid = pd.DataFrame(Y_hat_valid).transpose()
    Y_hat_test = pd.DataFrame(Y_hat_test).transpose()
    
    Y_hat_train.columns = ['label_hat']
    Y_hat_valid.columns = ['label_hat']
    Y_hat_test.columns = ['label_hat']
    
    
    confusionmatrix=confusion_matrix(y_test,Y_hat_test['label_hat'] )
    correct =  np.trace(confusionmatrix)
    total = confusionmatrix.sum()
    percent_accuracy = correct/total

    return (confusionmatrix,percent_accuracy)


K_NN_clf_result(best,X_train,y_train,X_valid,y_valid,X_test,y_test)

(array([[69,  0,  0],
        [ 0, 71,  8],
        [ 1,  0, 80]]),
 0.9606986899563319)